In [74]:
### PCA 1 번 잘한거지 모르겠습니다 피드백 주시면 수정하겠습니다.
rm(list=ls())

if(!require(rgl)) install.packages("rgl"); library(rgl) # scatter plot 을 3D로 표현하는 library

train = read.csv('data/Features_Variant_5.csv')
test = read.csv('data/Features_TestSet.csv')

names(train) <- paste0("X",as.character(seq(ncol(train))))
names(test) <- paste0("X",as.character(seq(ncol(test))))

train_scale <- scale(train[-c(38,54)])
test_scale <- scale(test[-c(38,54)])

train_scale <- cbind(train_scale, train$X54)
test_scale <- cbind(test_scale, test$X54)

train_scale <- data.frame(train_scale)
test_scale <- data.frame(test_scale)

##### 무슨이유인지 test data scale 하면 49,50번 열이 nan 입니다.
bad <- sapply(test_scale, function(x) all(is.nan(x)))

train_scale <- train_scale[ ,!bad]
test_scale <- test_scale[ ,!bad]

fit.con <- lm(V53~1, train_scale)
fit.full <- lm(V53~., train_scale)
fit.both <- step(fit.con, list(lower=fit.con, upper=fit.full), direction = "both")
summary(fit.both)

RMSE <- function(error) { sqrt(mean(error^2))}

RMSE(fit.both$residuals)

##############PCA 진행
pca_train <- prcomp(train_scale[-c(49)], center = TRUE) 
pca_test <- prcomp(test_scale[-c(49)], center = TRUE)

summary(pca_train)

train_PRC <- as.matrix(train_scale[-c(49)]) %*% pca_train$rotation
test_PRC <- as.matrix(test_scale[-c(49)]) %*% pca_test$rotation

trainF <- cbind(train_PRC, train$X54)
testF <- cbind(test_PRC, test$X54)

colnames(trainF)[49] <- "label"
colnames(testF)[49] <- "label"

trainF <- as.data.frame(trainF)

fit.con1 <- lm(label~1, trainF)
fit.full1 <- lm(label~., trainF)
fit.both1 <- step(fit.con1, list(lower=fit.con1, upper=fit.full1), direction = "both")
summary(fit.both1)

RMSE(fit.both1$residuals)

